In [11]:
import os
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt

def setup_eeg_montage(raw):
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage, match_case=False)
    raw.set_channel_types({ch: 'eeg' for ch in raw.ch_names})
    return raw

def process_edf_with_ica(edf_path):
    try:
        # Read the EDF file
        raw = mne.io.read_raw_edf(edf_path, preload=True)
        print(f"Channels found: {raw.ch_names}")
        
        # Set up the montage
        raw = setup_eeg_montage(raw)
        
        # Basic preprocessing
        raw.filter(l_freq=1.0, h_freq=40.0)
        
        # Set up and fit ICA
        ica = ICA(
            n_components=15,
            random_state=42,
            method='fastica',
            max_iter='auto'
        )
        
        # Fit ICA
        ica.fit(raw)
        print("ICA fitted successfully")
        
        # Create output directory
        output_filename = os.path.splitext(os.path.basename(edf_path))[0]
        output_dir = os.path.join("ica_outputs", output_filename)
        os.makedirs(output_dir, exist_ok=True)
        
        # Generate and save component plots
        fig = plt.figure(figsize=(12, 8))
        ica.plot_components(picks=range(15), show=False)
        plt.savefig(os.path.join(output_dir, 'ICA_components.png'))
        plt.close()
        print(f"Saved components plot to {output_dir}")
        
        # Generate and save sources plot
        fig = plt.figure(figsize=(15, 10))
        ica.plot_sources(raw, show=False)
        plt.savefig(os.path.join(output_dir, 'ICA_sources.png'))
        plt.close()
        print(f"Saved sources plot to {output_dir}")
        
        return ica
        
    except Exception as e:
        print(f"Error processing {os.path.basename(edf_path)}: {str(e)}")
        return None

def process_edf_folder(folder_path):
    # Ensure output directory exists
    os.makedirs("ica_outputs", exist_ok=True)
    
    # Get absolute path
    abs_folder_path = os.path.abspath(folder_path)
    print(f"Processing files from: {abs_folder_path}")
    
    results = {}
    
    # Process each EDF file
    for filename in os.listdir(abs_folder_path):
        if filename.endswith('.edf'):
            print(f"\nProcessing {filename}...")
            file_path = os.path.join(abs_folder_path, filename)
            
            ica = process_edf_with_ica(file_path)
            if ica is not None:
                results[filename] = ica
                print(f"Successfully processed {filename}")
            else:
                print(f"Failed to process {filename}")

    return results

if __name__ == "__main__":
    # Use the relative path
    edf_folder = "edf_dataset"
    
    # Enable debug logging
    mne.set_log_level('DEBUG')
    
    # Process files
    results = process_edf_folder(edf_folder)
    
    # Verify outputs
    output_dir = 'ica_outputs'
    if os.path.exists(output_dir):
        subdirs = os.listdir(output_dir)
        print("\nGenerated outputs:")
        for subdir in subdirs:
            subdir_path = os.path.join(output_dir, subdir)
            if os.path.isdir(subdir_path):
                files = os.listdir(subdir_path)
                print(f"{subdir}: {files}")

Processing files from: /Users/eznauc/EEG/depression-detection-eeg-python/edf_dataset

Processing S9_EO.edf...
Extracting EDF parameters from /Users/eznauc/EEG/depression-detection-eeg-python/edf_dataset/S9_EO.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 77311  =      0.000 ...   301.996 secs...
Channels found: ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1', 'EEG 23A-23R', 'EEG 24A-24R']
Using normal I/O
    Creating tag directory for /Users/eznauc/Library/Python/3.9/lib/python/site-packages/mne/data/fsaverage/fsaverage-fiducials.fif...
start { 0
    start { 107
        block = 107 nent = 4 nchild = 0
    end } 107
    block = 0 nent = 4 nchild = 1
end } 0
[done]
Error processing S9_EO.edf: DigMontage is only a subset

/var/folders/k3/wm9ghydj2r72xt_m7q60tpd40000gn/T/ipykernel_66145/876967424.py:9: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw.set_channel_types({ch: 'eeg' for ch in raw.ch_names})
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.5s.
ICA fitted successfully
Generating coords using: [0.    0.    0.    0.095]
Generating coords using: [0.    0.    0.    0.095]


/var/folders/k3/wm9ghydj2r72xt_m7q60tpd40000gn/T/ipykernel_66145/876967424.py:33: RuntimeWarning: Using n_components=15 (resulting in n_components_=15) may lead to an unstable mixing matrix estimation because the ratio between the largest (20) and smallest (1.6e-17) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 1
  ica.fit(raw)


Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean


...

Fitting ICA took 0.6s.
ICA fitted successfully
Generating coords using: [0.    0.    0.    0.095]
Generating coords using: [0.    0.    0.    0.095]


/var/folders/k3/wm9ghydj2r72xt_m7q60tpd40000gn/T/ipykernel_66145/876967424.py:33: RuntimeWarning: Using n_components=15 (resulting in n_components_=15) may lead to an unstable mixing matrix estimation because the ratio between the largest (20) and smallest (2e-17) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 1
  ica.fit(raw)


Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (20,)
Interpolation mode cubic, extrapolation mode head to mean


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 0.8s.
ICA fitted successfully
Generating coords using: [0.    0.    0.    0.095]
Generating coords using: [0.    0.    0.    0.095]
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpolation mode cubic, extrapolation mode head to mean
Plotting topomap for eeg data shape (19,)
Interpola

In [ ]:
pip install mne numpy matplotlib

In [9]:
import os
print(f"Current working directory: {os.getcwd()}")

Current working directory: /Users/eznauc/EEG/depression-detection-eeg-python


In [10]:
try:
    os.makedirs("test_dir", exist_ok=True)
    os.rmdir("test_dir")
    print("Write permissions OK")
except Exception as e:
    print(f"Permission error: {e}")

Write permissions OK
